# Use extracted wiki + bank texts to train embeddings (ft)

In [1]:
import gensim
import re
import nltk
import multiprocessing
path2 = '/opt/apps/cda/ez/sb_sandboxpuk/sandboxpuk_scripte/wiki/'

In [2]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [3]:
#list(text_preprocessed)[:3]

### Loading wiki text corpus


In [4]:
%%time
wiki_txt = open(path2 + 'wiki_bank_first.txt', 'r')

class txt_wiki_all(object):
    def __iter__(self):
        for sen in wiki_txt:
            
            sen = re.sub('[^A-Za-zÄÖÜßäöüß]+', ' ',sen)
            yield [a.lower() for a in sen.split()]

CPU times: user 64 µs, sys: 138 µs, total: 202 µs
Wall time: 120 µs


### Train plain wiki w2v

In [ ]:
%%time
from gensim.models.word2vec import Word2Vec
name = 'wiki_w2v_300.model'

params = {'size': 300, 'window': 10, 'min_count': 50, 'iter':30,'max_vocab_size':500000,
          'workers': max(1, 10 #multiprocessing.cpu_count() - 1
                        ), 'sample': 1e-3,}
word2vec = Word2Vec(sentences=list(text_preprocessed), **params)
word2vec.save(path2 + name)

In [ ]:
word_vectors = word2vec.wv

from gensim.test.utils import get_tmpfile

fname = get_tmpfile("vectors.kv")
print(fname)
word_vectors.save("vectors.kv")

In [ ]:
word2vec.wv.most_similar('karte')

In [ ]:
word2vec.most_similar(positive=['frau', 'könig'], negative=['mann'], topn=10)

In [ ]:
path='/opt/apps/cda/ez/sb_sandboxpuk/sandboxpuk_scripte/wiki/'
model = 'wiki_w2v_300.model'
from gensim.models.word2vec import Word2Vec
word2vec = Word2Vec.load(path+model)


In [ ]:
import re
kuko_text = []
path = '/var/iophome/cb2lehk/feedbacks_v02/'

with open(path+'data/KuKo2.txt') as fin:
    for line in fin:
        #print(line)
        line = re.sub('[^A-Za-zÄÖÜßäöü]+', ' ',line)
        line = [word.lower() for word in line.split()]
        kuko_text.append(line)
        
nps_text = []
with open(path+'data/NPS_LERNDATEN_FEEDBACK2KATEGORIE.txt') as fin:
    for line in fin:
        #print(line)
        line = re.sub('[^A-Za-zÄÖÜßäöü]+', ' ',line.split('/t')[0])
        line = [word.lower() for word in line.split()]
        nps_text.append(line)
#nps_text[1:2]
data = spark.sql("select csp_feedbackaboutclientpotential from \
                 az_01_01_38_dynamics.csp_salespotential_v001 \
                 where length(csp_feedbackaboutclientpotential) > 2").toPandas()

data = data['csp_feedbackaboutclientpotential'].tolist()

feedback_text = []

for line in data:
        #print(line)
    line = re.sub('[^A-Za-zÄÖÜßäöü]+', ' ',line)
    line = [word.lower() for word in line.split()]
    feedback_text.append(line)

In [ ]:
import re
text_all = []
path = '/var/iophome/cb2lehk/feedbacks_v02/'

with open('data/kuko01.txt') as fin:
    for line in fin:
        #print(line)
        line = re.sub('[^A-Za-zÄÖÜßäöü]+', ' ',line)
        line = [word.lower() for word in line.split()]
        text_all.append(line)
        
with open('data/kuko02.txt') as fin:
    for line in fin:
        #print(line)
        line = re.sub('[^A-Za-zÄÖÜßäöü]+', ' ',line)
        line = [word.lower() for word in line.split()]
        text_all.append(line)

with open('data/kuko03.txt') as fin:
    for line in fin:
        #print(line)
        line = re.sub('[^A-Za-zÄÖÜßäöü]+', ' ',line)
        line = [word.lower() for word in line.split()]
        text_all.append(line)

        
with open('data/nps_text082019.txt') as fin:
    for line in fin:
        #print(line)
        line = re.sub('[^A-Za-zÄÖÜßäöü]+', ' ',line)
        line = [word.lower() for word in line.split()]
        text_all.append(line)
        

In [ ]:
text = nps_text[1:] + kuko_text[1:] + feedback_text

In [ ]:
word2vec.build_vocab(text_all, update=True)

In [ ]:
word2vec.train(text_all,len(text_all),epochs=10)

In [ ]:
nps = {}
notin = {}
with open('data/nps_text082019.txt') as fin:
    for line in fin:
        #print(line)
        line = re.sub('[^A-Za-zÄÖÜßäöü]+', ' ',line)
        line = [word.lower() for word in line.split()]
        for word in line:
            if word not in word2vec.wv.vocab:
                if word in notin.keys():
                    notin[word] = notin[word] + 1
                else:
                    notin[word] = 1
            else:
                if word in nps.keys():
                    nps[word] = nps[word] + 1
                else:
                    nps[word] = 1



In [ ]:
nps['möglichst']

In [ ]:
for key, value in sorted(notin.items(), key=lambda item: item[1])[-10:]:
    print("%s: %s" % (key, value))

In [ ]:
path2 = '/opt/apps/cda/ez/sb_sandboxpuk/sandboxpuk_scripte/wiki/'
word2vec.save(path2 + model)

In [ ]:
keep2 = ['zu', 'um', 'ab', 'im','da', 'er', 'an', 'in', 'dr', 'so','es','am','ob','cc']
vocab2 = [word for word in word2vec.wv.vocab if len(word) == 2 and  word not in keep2]

In [ ]:
vocab_to_emb = {}
for word in word2vec.wv.vocab:
    if word not in vocab2:
        vocab_to_emb[word] = word2vec.wv[word]

In [ ]:
import pickle
path = '/opt/apps/cda/ez/sb_sandboxpuk/sandboxpuk_scripte/textmining_03/data/books/'

with open(path+'vocab_to_emb_w2v_300.p', 'wb') as handle:
    pickle.dump(vocab_to_emb, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open(path+'nps_vocab.p', 'wb') as handle:
    pickle.dump(nps, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
import embedding
embedding.model_to_books(model_embed)

# FT

In [5]:
%%time
from gensim.models import FastText
name = 'wiki_ft_300.model'
#sentences = txt_wiki_all()

params = {'size': 300, 'window': 4, 'iter':100,'word_ngrams':1,'sg':1,'max_vocab_size':500000,
          'workers': max(1, 8 #multiprocessing.cpu_count() - 1
                        ), 'sample': 1e-3,}
#ft = FastText(sentences = txt_wiki_all(), **params)

ft = FastText(**params)
ft.build_vocab(txt_wiki_all())
ft.train(txt_wiki_all(), total_examples=ft.corpus_count, epochs=20)

ft.save(path2 + name)

2019-12-04 18:31:55,491 : INFO : collecting all words and their counts
2019-12-04 18:31:55,492 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-12-04 18:31:55,636 : INFO : PROGRESS: at sentence #10000, processed 105525 words, keeping 9699 word types
2019-12-04 18:31:55,779 : INFO : PROGRESS: at sentence #20000, processed 210250 words, keeping 14551 word types
2019-12-04 18:31:55,916 : INFO : PROGRESS: at sentence #30000, processed 314166 words, keeping 17960 word types
2019-12-04 18:31:56,055 : INFO : PROGRESS: at sentence #40000, processed 420151 words, keeping 17960 word types
2019-12-04 18:31:56,195 : INFO : PROGRESS: at sentence #50000, processed 524837 words, keeping 17960 word types
2019-12-04 18:31:56,334 : INFO : PROGRESS: at sentence #60000, processed 628902 words, keeping 17960 word types
2019-12-04 18:31:56,479 : INFO : PROGRESS: at sentence #70000, processed 734878 words, keeping 17960 word types
2019-12-04 18:31:56,620 : INFO : PROGRESS: at s

2019-12-04 18:32:05,652 : INFO : PROGRESS: at sentence #720000, processed 7550108 words, keeping 17960 word types
2019-12-04 18:32:05,791 : INFO : PROGRESS: at sentence #730000, processed 7656055 words, keeping 17960 word types
2019-12-04 18:32:05,930 : INFO : PROGRESS: at sentence #740000, processed 7760535 words, keeping 17960 word types
2019-12-04 18:32:06,069 : INFO : PROGRESS: at sentence #750000, processed 7865137 words, keeping 17960 word types
2019-12-04 18:32:06,208 : INFO : PROGRESS: at sentence #760000, processed 7970139 words, keeping 17960 word types
2019-12-04 18:32:06,351 : INFO : PROGRESS: at sentence #770000, processed 8074970 words, keeping 17960 word types
2019-12-04 18:32:06,491 : INFO : PROGRESS: at sentence #780000, processed 8180048 words, keeping 17960 word types
2019-12-04 18:32:06,631 : INFO : PROGRESS: at sentence #790000, processed 8284889 words, keeping 17960 word types
2019-12-04 18:32:06,771 : INFO : PROGRESS: at sentence #800000, processed 8389571 words,

2019-12-04 18:32:15,853 : INFO : PROGRESS: at sentence #1440000, processed 15106801 words, keeping 19936 word types
2019-12-04 18:32:16,006 : INFO : PROGRESS: at sentence #1450000, processed 15224879 words, keeping 22916 word types
2019-12-04 18:32:16,142 : INFO : PROGRESS: at sentence #1460000, processed 15327721 words, keeping 24094 word types
2019-12-04 18:32:16,272 : INFO : PROGRESS: at sentence #1470000, processed 15426969 words, keeping 24932 word types
2019-12-04 18:32:16,409 : INFO : PROGRESS: at sentence #1480000, processed 15529298 words, keeping 25665 word types
2019-12-04 18:32:16,551 : INFO : PROGRESS: at sentence #1490000, processed 15633776 words, keeping 26447 word types
2019-12-04 18:32:16,681 : INFO : PROGRESS: at sentence #1500000, processed 15728489 words, keeping 27331 word types
2019-12-04 18:32:16,810 : INFO : PROGRESS: at sentence #1510000, processed 15828857 words, keeping 28508 word types
2019-12-04 18:32:16,935 : INFO : PROGRESS: at sentence #1520000, process

2019-12-04 18:32:25,956 : INFO : PROGRESS: at sentence #2150000, processed 22733297 words, keeping 59596 word types
2019-12-04 18:32:26,116 : INFO : PROGRESS: at sentence #2160000, processed 22854423 words, keeping 59596 word types
2019-12-04 18:32:26,281 : INFO : PROGRESS: at sentence #2170000, processed 22974608 words, keeping 59596 word types
2019-12-04 18:32:26,439 : INFO : PROGRESS: at sentence #2180000, processed 23095510 words, keeping 59596 word types
2019-12-04 18:32:26,599 : INFO : PROGRESS: at sentence #2190000, processed 23218235 words, keeping 59596 word types
2019-12-04 18:32:26,750 : INFO : PROGRESS: at sentence #2200000, processed 23335031 words, keeping 59596 word types
2019-12-04 18:32:26,889 : INFO : PROGRESS: at sentence #2210000, processed 23442340 words, keeping 59596 word types
2019-12-04 18:32:27,020 : INFO : PROGRESS: at sentence #2220000, processed 23541459 words, keeping 59596 word types
2019-12-04 18:32:27,152 : INFO : PROGRESS: at sentence #2230000, process

2019-12-04 18:32:36,037 : INFO : PROGRESS: at sentence #2860000, processed 30393024 words, keeping 59596 word types
2019-12-04 18:32:36,184 : INFO : PROGRESS: at sentence #2870000, processed 30506503 words, keeping 59596 word types
2019-12-04 18:32:36,333 : INFO : PROGRESS: at sentence #2880000, processed 30619744 words, keeping 59596 word types
2019-12-04 18:32:36,481 : INFO : PROGRESS: at sentence #2890000, processed 30731804 words, keeping 59596 word types
2019-12-04 18:32:36,632 : INFO : PROGRESS: at sentence #2900000, processed 30846317 words, keeping 59596 word types
2019-12-04 18:32:36,785 : INFO : PROGRESS: at sentence #2910000, processed 30963700 words, keeping 59596 word types
2019-12-04 18:32:36,942 : INFO : PROGRESS: at sentence #2920000, processed 31084267 words, keeping 59596 word types
2019-12-04 18:32:37,103 : INFO : PROGRESS: at sentence #2930000, processed 31204922 words, keeping 59596 word types
2019-12-04 18:32:37,263 : INFO : PROGRESS: at sentence #2940000, process

...

2019-12-04 18:55:21,284 : INFO : PROGRESS: at sentence #70340000, processed 1057318191 words, keeping 376822 word types
2019-12-04 18:55:21,421 : INFO : PROGRESS: at sentence #70350000, processed 1057423961 words, keeping 376822 word types
2019-12-04 18:55:21,562 : INFO : PROGRESS: at sentence #70360000, processed 1057530304 words, keeping 376822 word types
2019-12-04 18:55:21,708 : INFO : PROGRESS: at sentence #70370000, processed 1057637431 words, keeping 376822 word types
2019-12-04 18:55:21,853 : INFO : PROGRESS: at sentence #70380000, processed 1057744909 words, keeping 376822 word types
2019-12-04 18:55:21,989 : INFO : PROGRESS: at sentence #70390000, processed 1057851641 words, keeping 376822 word types
2019-12-04 18:55:22,127 : INFO : PROGRESS: at sentence #70400000, processed 1057960530 words, keeping 376822 word types
2019-12-04 18:55:22,266 : INFO : PROGRESS: at sentence #70410000, processed 1058070386 words, keeping 376822 word types
2019-12-04 18:55:22,408 : INFO : PROGRES

2019-12-04 18:57:14,151 : INFO : worker thread finished; awaiting finish of 0 more threads
2019-12-04 18:57:14,151 : INFO : EPOCH - 1 : training on 0 raw words (0 effective words) took 0.0s, 0 effective words/s
2019-12-04 18:57:14,152 : WARNING : EPOCH - 1 : supplied example count (0) did not equal expected count (70868669)
2019-12-04 18:57:14,156 : WARNING : train() called with an empty iterator (if not intended, be sure to provide a corpus that offers restartable iteration = an iterable).
2019-12-04 18:57:14,158 : INFO : worker thread finished; awaiting finish of 7 more threads
2019-12-04 18:57:14,158 : INFO : worker thread finished; awaiting finish of 6 more threads
2019-12-04 18:57:14,159 : INFO : worker thread finished; awaiting finish of 5 more threads
2019-12-04 18:57:14,159 : INFO : worker thread finished; awaiting finish of 4 more threads
2019-12-04 18:57:14,160 : INFO : worker thread finished; awaiting finish of 3 more threads
2019-12-04 18:57:14,160 : INFO : worker thread fi

2019-12-04 18:57:14,238 : WARNING : train() called with an empty iterator (if not intended, be sure to provide a corpus that offers restartable iteration = an iterable).
2019-12-04 18:57:14,239 : INFO : worker thread finished; awaiting finish of 7 more threads
2019-12-04 18:57:14,240 : INFO : worker thread finished; awaiting finish of 6 more threads
2019-12-04 18:57:14,240 : INFO : worker thread finished; awaiting finish of 5 more threads
2019-12-04 18:57:14,241 : INFO : worker thread finished; awaiting finish of 4 more threads
2019-12-04 18:57:14,241 : INFO : worker thread finished; awaiting finish of 3 more threads
2019-12-04 18:57:14,242 : INFO : worker thread finished; awaiting finish of 2 more threads
2019-12-04 18:57:14,242 : INFO : worker thread finished; awaiting finish of 1 more threads
2019-12-04 18:57:14,243 : INFO : worker thread finished; awaiting finish of 0 more threads
2019-12-04 18:57:14,244 : INFO : EPOCH - 9 : training on 0 raw words (0 effective words) took 0.0s, 0 

2019-12-04 18:57:14,322 : INFO : worker thread finished; awaiting finish of 6 more threads
2019-12-04 18:57:14,323 : INFO : worker thread finished; awaiting finish of 5 more threads
2019-12-04 18:57:14,323 : INFO : worker thread finished; awaiting finish of 4 more threads
2019-12-04 18:57:14,324 : INFO : worker thread finished; awaiting finish of 3 more threads
2019-12-04 18:57:14,324 : INFO : worker thread finished; awaiting finish of 2 more threads
2019-12-04 18:57:14,325 : INFO : worker thread finished; awaiting finish of 1 more threads
2019-12-04 18:57:14,326 : INFO : worker thread finished; awaiting finish of 0 more threads
2019-12-04 18:57:14,326 : INFO : EPOCH - 16 : training on 0 raw words (0 effective words) took 0.0s, 0 effective words/s
2019-12-04 18:57:14,327 : WARNING : EPOCH - 16 : supplied example count (0) did not equal expected count (70868669)
2019-12-04 18:57:14,331 : WARNING : train() called with an empty iterator (if not intended, be sure to provide a corpus that o

CPU times: user 25min 30s, sys: 23.9 s, total: 25min 54s
Wall time: 25min 43s


In [ ]:
from gensim.models import FastText as ft
ft = ft.load('data/wiki_ft_300.model')

In [6]:
ft.wv.most_similar('karte')

2019-12-04 18:57:38,765 : INFO : precomputing L2-norms of word weight vectors
2019-12-04 18:57:39,430 : INFO : precomputing L2-norms of ngram weight vectors
/opt/apps/anaconda3/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('kkarte', 0.6204562783241272),
 ('eckarte', 0.5813156366348267),
 ('kartei', 0.5756651759147644),
 ('karten', 0.5659630298614502),
 ('ihrekarte', 0.5561856031417847),
 ('karteikarten', 0.536436140537262),
 ('zahlkarte', 0.5272395014762878),
 ('neuekarte', 0.5252927541732788),
 ('bahnkarte', 0.5089269876480103),
 ('seekarte', 0.5052211284637451)]

In [8]:
ft.most_similar(positive=['mann', 'könig'], negative=['frau'], topn=10)

/opt/apps/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  if __name__ == '__main__':
/opt/apps/anaconda3/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('königin', 0.39161479473114014),
 ('königsmarck', 0.3716628849506378),
 ('könige', 0.36920976638793945),
 ('königl', 0.36845505237579346),
 ('königen', 0.3553083539009094),
 ('königs', 0.3526017963886261),
 ('königsplatz', 0.3464611768722534),
 ('könemann', 0.3417457044124603),
 ('königsgut', 0.34161829948425293),
 ('königsmacher', 0.3284785747528076)]

In [ ]:
word_vectors = word2vec.wv

from gensim.test.utils import get_tmpfile
from gensim.models import KeyedVectors

fname = get_tmpfile("vectors.kv")
word_vectors.save(fname)

In [ ]:
word_vectors = KeyedVectors.load(fname, mmap='r')

In [ ]:
import nltk
path2 = '/opt/apps/cda/ez/sb_sandboxpuk/sandboxpuk_scripte/wiki/data/'

with open (path2+'wiki_.txt', 'rt') as myfile:  # Open file lorem.txt for reading text
    i=1
    a = []
    for myline in myfile:                 # For each line, read it to a string 
        i+=1
        a.append(myline)
        if i < 6:
            print(myline)  
a

In [ ]:
a[-8]